# Exercise for the part 4

For this exercise you will use the WRF-ARW output file: `wrf_filtered.nc`.

The WRF-ARW model is a mesoscale numerical weather prediction system 
designed for both atmospheric research and operational forecasting 
applications.

**Part A:**

Plot the 2-m temperature field on 3 Dec 2015 at 15:00
using 16 contour levels:
* on the equidistant cylindrical projection
* on the native Lambert Conformal projection

In this exercise you can use the `XLAT` and `XLONG` 
2D-arrays providing the coordinates for each point.

**Part B:**

Plot again the temperature field in the native grid 
using the projection information specified as global 
attributes (i.e., don't use `XLAT` and `XLONG`).

For this exercise use ax.contourf and ax.imshow and
compare the performances printing the time of the 
execution.

_Hints:_
* Use the gobal attributes to define the Lambert projection: CEN_LON, CEN_LAT, TRUELAT1, TRUELAT2, DX, DY
* In part B, set the parameter `extent`

In [ ]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import os

In [ ]:
path   = "."
fname  = "wrf_filtered.nc"
ncfile = os.path.join(path,fname)

In [ ]:
ds = xr.open_dataset(ncfile)
print(ds.keys())

In [ ]:
ds=ds.rename({'XTIME':'Time'})
ds.T2.sel(Time='2015-12-3 15:00')

In [ ]:
prj_ce  = ccrs.PlateCarree()

In [ ]:
%%time
ax = plt.axes(projection=prj_ce)
ax.coastlines()
ax.gridlines(draw_labels=True)
ds.T2.sel(Time='2015-12-3 15:00').plot.contourf('XLONG','XLAT',ax=ax,levels=16)

In [ ]:
prj_lcc = ccrs.LambertConformal(central_longitude  = ds.attrs['CEN_LON'], 
                                central_latitude   = ds.attrs['CEN_LAT'], 
                                standard_parallels = (ds.attrs['TRUELAT1'],ds.attrs['TRUELAT2']), 
                               )

In [ ]:
%%time
ax = plt.axes(projection=prj_lcc)
ax.coastlines()
ax.gridlines(draw_labels=True)
ds.T2.sel(Time='2015-12-3 15:00').plot.contourf('XLONG','XLAT',
                                                ax=ax,
                                                transform = prj_ce,
                                                levels=16,
                                               )

In [ ]:
LL_lat = ds.isel(Time=0,south_north=0,west_east=0).XLAT.values
LL_lon = ds.isel(Time=0,south_north=0,west_east=0).XLONG.values
UR_lat = ds.isel(Time=0,south_north=-1,west_east=-1).XLAT.values
UR_lon = ds.isel(Time=0,south_north=-1,west_east=-1).XLONG.values

#Upper right
UR = (UR_lon,UR_lat)

#Lower left
LL = (LL_lon,LL_lat)

In [ ]:
NY = ds.sizes['south_north']
NX = ds.sizes['west_east']

X_max = NX*ds.attrs['DX']
Y_max = NY*ds.attrs['DY']
extent = [-0.5*X_max,0.5*X_max,-0.5*Y_max,0.5*Y_max]

In [ ]:
%%time
ax = plt.axes(projection=prj_lcc)
ax.coastlines()
ax.gridlines(draw_labels=True)
data = ds.T2.sel(Time='2015-12-3 15:00').values

ax.contourf(data,
            extent = extent,
            transform = prj_lcc,
            levels=16
           )
#ax.set_extent([0,40,20,55],crs=prj_ce)

In [ ]:
%%time
ax = plt.axes(projection=prj_lcc)
ax.coastlines()
ax.gridlines(draw_labels=True)
data = ds.T2.sel(Time='2015-12-3 15:00').values

ax.imshow(data,extent = extent)

In [ ]:
ds.close()